# LTX-Video - Generation Video Rapide et Legere

**Module :** 02-Video-Advanced  
**Niveau :** Intermediaire  
**Technologies :** LTX-Video (Lightricks), diffusers  
**Duree estimee :** 45 minutes  
**VRAM :** ~8 GB  

## Objectifs d'Apprentissage

- [ ] Comprendre l'architecture LTX-Video et son approche legere
- [ ] Charger le pipeline LTX-Video via diffusers
- [ ] Generer des videos text-to-video rapidement
- [ ] Animer une image statique (image-to-video)
- [ ] Appliquer du style transfer video-to-video
- [ ] Comparer les compromis vitesse/qualite avec HunyuanVideo

## Prerequis

- GPU avec 8+ GB VRAM (RTX 3060 8GB minimum)
- Notebook 02-1 (HunyuanVideo) complete pour comparaison
- Packages : `diffusers>=0.32`, `transformers`, `torch`, `accelerate`, `imageio`, `imageio-ffmpeg`

**Navigation :** [<< 02-1](02-1-HunyuanVideo-Generation.ipynb) | [Index](../README.md) | [Suivant >>](02-3-Wan-Video-Generation.ipynb)

In [ ]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres modele
model_id = "Lightricks/LTX-Video"  # Modele LTX-Video
device = "cuda"                    # Device de calcul

# Parametres generation
num_frames = 16                    # Nombre de frames a generer
num_inference_steps = 20           # Nombre d'etapes de debruitage
guidance_scale = 7.5               # CFG scale (adherence au prompt)
height = 480                       # Hauteur video
width = 704                        # Largeur video
fps_output = 8                     # FPS de la video de sortie

# Configuration
run_generation = True              # Executer la generation
run_img2vid = True                 # Tester image-to-video
save_as_mp4 = True                 # Sauvegarder en MP4
save_results = True

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import time
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import logging

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging
        print("Helpers GenAI importes")
    except ImportError:
        print("Helpers GenAI non disponibles - mode autonome")

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'ltx_video'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('ltx_video')

print(f"LTX-Video - Generation Video Rapide et Legere")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Frames : {num_frames}, Steps : {num_inference_steps}, CFG : {guidance_scale}")

In [ ]:
# Chargement .env et verification GPU
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

# Verification GPU
print("\n--- VERIFICATION GPU ---")
print("=" * 40)

import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_total = torch.cuda.get_device_properties(0).total_mem / 1024**3
    vram_free = (torch.cuda.get_device_properties(0).total_mem - torch.cuda.memory_allocated(0)) / 1024**3
    
    print(f"GPU : {gpu_name}")
    print(f"VRAM totale : {vram_total:.1f} GB")
    print(f"VRAM libre : {vram_free:.1f} GB")
    print(f"CUDA : {torch.version.cuda}")
    
    if vram_total < 8:
        print(f"\nAttention : VRAM ({vram_total:.0f} GB) < 8 GB recommandes")
        height = 320
        width = 480
        num_frames = 8
        print(f"  Resolution reduite a {width}x{height}, {num_frames} frames")
else:
    print("CUDA non disponible.")
    print("LTX-Video necessite un GPU. Le notebook montrera le code sans executer.")
    run_generation = False
    run_img2vid = False
    device = "cpu"

# Verification des dependances
print("\n--- VERIFICATION DEPENDANCES ---")
print("=" * 40)

deps_ok = True

try:
    import diffusers
    print(f"diffusers : v{diffusers.__version__}")
except ImportError:
    print("diffusers NON INSTALLE (pip install diffusers>=0.32)")
    deps_ok = False

try:
    import transformers
    print(f"transformers : v{transformers.__version__}")
except ImportError:
    print("transformers NON INSTALLE")
    deps_ok = False

try:
    import imageio
    print(f"imageio : v{imageio.__version__}")
except ImportError:
    print("imageio NON INSTALLE")
    deps_ok = False

if not deps_ok:
    print("\nDependances manquantes. Le notebook montrera le code sans executer.")
    run_generation = False
    run_img2vid = False

print(f"\nDevice : {device}")
print(f"Generation activee : {run_generation}")
print(f"Image-to-video activee : {run_img2vid}")

## Section 1 : Architecture LTX-Video

LTX-Video est un modele de generation video developpe par Lightricks, concu pour etre
rapide et leger tout en maintenant une qualite acceptable. Il supporte trois modes :
text-to-video, image-to-video et video-to-video.

| Composant | Description |
|-----------|-------------|
| **Architecture** | DiT (Diffusion Transformer) avec compression spatio-temporelle |
| **VAE** | Video VAE avec facteur de compression 1:192 |
| **Text encoder** | T5-XXL pour la comprehension des prompts |
| **Scheduler** | Flow matching pour un debruitage efficace |

### Avantages de LTX-Video

| Aspect | LTX-Video | HunyuanVideo (02-1) |
|--------|-----------|---------------------|
| VRAM requise | ~8 GB | ~18 GB |
| Vitesse | 2-5x plus rapide | Reference |
| Modes | Text/Image/Video | Text uniquement |
| Qualite | Bonne | Tres bonne |
| Resolution | Jusqu'a 720x480 | Jusqu'a 720p |

In [ ]:
# Chargement du pipeline LTX-Video
pipe_t2v = None

if run_generation:
    print("\n--- CHARGEMENT DU PIPELINE ---")
    print("=" * 40)
    
    try:
        from diffusers import LTXPipeline
        from diffusers.utils import export_to_video
        
        start_load = time.time()
        
        print(f"Chargement modele : {model_id}")
        
        pipe_t2v = LTXPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16
        )
        pipe_t2v = pipe_t2v.to(device)
        
        # Optimisations memoire
        pipe_t2v.enable_vae_slicing()
        try:
            pipe_t2v.enable_model_cpu_offload()
        except Exception:
            pass
        
        load_time = time.time() - start_load
        
        if device == "cuda":
            vram_used = torch.cuda.memory_allocated(0) / 1024**3
            print(f"  VRAM utilisee : {vram_used:.1f} GB")
        
        print(f"Pipeline text-to-video charge en {load_time:.1f}s")
        print(f"  VAE slicing : actif")
        print(f"  Resolution : {width}x{height}")
        
    except Exception as e:
        print(f"Erreur chargement pipeline : {type(e).__name__}: {str(e)[:200]}")
        print("Le notebook continuera sans generation.")
        run_generation = False
        pipe_t2v = None
else:
    print("Chargement pipeline desactive")

## Section 2 : Generation text-to-video

LTX-Video excelle dans la generation rapide de videos courtes a partir de prompts textuels.
Son architecture legere permet des iterations plus rapides que HunyuanVideo.

In [ ]:
# Generation text-to-video
print("\n--- GENERATION TEXT-TO-VIDEO ---")
print("=" * 40)

def generate_ltx_video(prompt: str, negative_prompt: str = "",
                       seed: int = 42, pipeline=None) -> Dict[str, Any]:
    """
    Genere une video avec LTX-Video.
    
    Args:
        prompt: Description textuelle de la video
        negative_prompt: Elements a eviter
        seed: Graine aleatoire pour reproductibilite
        pipeline: Pipeline a utiliser (defaut: pipe_t2v)
    
    Returns:
        Dict avec frames, temps de generation et metadonnees
    """
    active_pipe = pipeline or pipe_t2v
    if active_pipe is None:
        return {"success": False, "error": "Pipeline non charge"}
    
    try:
        generator = torch.Generator(device=device).manual_seed(seed)
        
        if device == "cuda":
            torch.cuda.reset_peak_memory_stats()
        
        start_time = time.time()
        
        output = active_pipe(
            prompt=prompt,
            negative_prompt=negative_prompt or "low quality, blurry, distorted, worst quality",
            num_frames=num_frames,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            height=height,
            width=width,
            generator=generator
        )
        
        gen_time = time.time() - start_time
        frames = output.frames[0]
        
        result = {
            "success": True,
            "frames": frames,
            "generation_time": gen_time,
            "time_per_frame": gen_time / num_frames,
            "prompt": prompt,
            "seed": seed,
            "params": {
                "num_frames": num_frames,
                "guidance_scale": guidance_scale,
                "num_inference_steps": num_inference_steps,
                "height": height,
                "width": width
            }
        }
        
        if device == "cuda":
            result["vram_peak"] = torch.cuda.max_memory_allocated(0) / 1024**3
        
        return result
        
    except Exception as e:
        return {"success": False, "error": f"{type(e).__name__}: {str(e)[:200]}"}


# Premier test
prompt_1 = "a butterfly landing on a colorful flower in a garden, macro photography, soft bokeh background, natural lighting"

if run_generation:
    print(f"Prompt : {prompt_1}")
    print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
    print(f"Resolution : {width}x{height}")
    print(f"\nGeneration en cours...")
    
    result_1 = generate_ltx_video(prompt_1, seed=42)
    
    if result_1['success']:
        frames = result_1['frames']
        print(f"\nGeneration reussie")
        print(f"  Temps total : {result_1['generation_time']:.1f}s")
        print(f"  Temps/frame : {result_1['time_per_frame']:.1f}s")
        print(f"  Frames : {len(frames)}")
        if 'vram_peak' in result_1:
            print(f"  VRAM pic : {result_1['vram_peak']:.1f} GB")
        
        # Affichage en grille
        n_display = min(8, len(frames))
        indices = np.linspace(0, len(frames) - 1, n_display, dtype=int)
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        axes_flat = axes.flatten()
        for i, idx in enumerate(indices):
            if i < len(axes_flat):
                axes_flat[i].imshow(frames[idx])
                axes_flat[i].set_title(f"Frame {idx + 1}/{len(frames)}", fontsize=9)
                axes_flat[i].axis('off')
        for i in range(len(indices), len(axes_flat)):
            axes_flat[i].axis('off')
        plt.suptitle(f"LTX-Video : {prompt_1[:60]}...", fontsize=11, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        if save_as_mp4:
            mp4_path = OUTPUT_DIR / "ltx_demo.mp4"
            export_to_video(frames, str(mp4_path), fps=fps_output)
            mp4_size_kb = mp4_path.stat().st_size / 1024
            print(f"  MP4 sauvegarde : {mp4_path.name} ({mp4_size_kb:.1f} KB)")
    else:
        print(f"Erreur : {result_1['error']}")
else:
    print("Generation desactivee")
    print(f"\nExemple de code pour generer :")
    print(f"  result = generate_ltx_video('{prompt_1[:50]}...', seed=42)")

### Interpretation : Text-to-video LTX

| Aspect | Valeur typique | Comparaison HunyuanVideo |
|--------|---------------|-------------------------|
| Temps total | 10-30s (RTX 3090) | 2-5x plus rapide |
| VRAM pic | 6-8 GB | 2-3x moins gourmand |
| Qualite | Bonne | Legerement inferieure |
| Coherence temporelle | Bonne | Comparable |

**Points cles** :
1. LTX-Video est ideal pour le prototypage rapide et les iterations
2. La qualite est suffisante pour la plupart des cas d'usage non-production
3. Le facteur de compression VAE (1:192) explique la vitesse elevee

## Section 3 : Image-to-video (animation d'image)

LTX-Video peut animer une image statique en video coherente. Cela permet de donner
vie a des photos, illustrations ou images generees par des modeles text-to-image.

In [ ]:
# Image-to-video : animation d'image statique
if run_img2vid:
    print("\n--- IMAGE-TO-VIDEO ---")
    print("=" * 40)
    
    try:
        from diffusers import LTXImageToVideoPipeline
        from diffusers.utils import export_to_video
        
        # Creer une image de test (ou charger une existante)
        print("Creation d'une image de test...")
        test_img = Image.new('RGB', (width, height), (100, 150, 200))
        from PIL import ImageDraw
        draw = ImageDraw.Draw(test_img)
        
        # Dessiner un paysage simple
        # Ciel gradient
        for y in range(height // 2):
            r = int(100 + 50 * y / (height // 2))
            g = int(150 + 50 * y / (height // 2))
            b = int(220 - 20 * y / (height // 2))
            draw.line([(0, y), (width, y)], fill=(r, g, b))
        # Sol vert
        draw.rectangle([0, height // 2, width, height], fill=(80, 140, 60))
        # Soleil
        draw.ellipse([width - 120, 30, width - 40, 110], fill=(255, 220, 100))
        
        test_img_path = OUTPUT_DIR / "test_input_image.png"
        test_img.save(str(test_img_path))
        print(f"Image de test sauvegardee : {test_img_path.name}")
        
        # Charger le pipeline image-to-video
        print(f"\nChargement pipeline image-to-video...")
        pipe_i2v = LTXImageToVideoPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16
        )
        pipe_i2v = pipe_i2v.to(device)
        pipe_i2v.enable_vae_slicing()
        
        # Generer la video a partir de l'image
        i2v_prompt = "the landscape comes alive, clouds moving slowly, grass swaying in the wind, golden hour lighting"
        
        print(f"Prompt : {i2v_prompt}")
        print(f"Generation image-to-video en cours...")
        
        generator = torch.Generator(device=device).manual_seed(42)
        if device == "cuda":
            torch.cuda.reset_peak_memory_stats()
        
        start_time = time.time()
        output_i2v = pipe_i2v(
            image=test_img,
            prompt=i2v_prompt,
            negative_prompt="low quality, blurry, distorted",
            num_frames=num_frames,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            generator=generator
        )
        i2v_time = time.time() - start_time
        
        i2v_frames = output_i2v.frames[0]
        
        if device == "cuda":
            vram_i2v = torch.cuda.max_memory_allocated(0) / 1024**3
            print(f"  VRAM pic : {vram_i2v:.1f} GB")
        
        print(f"Generation reussie en {i2v_time:.1f}s")
        print(f"  Frames : {len(i2v_frames)}")
        
        # Affichage : image source + frames generees
        fig, axes = plt.subplots(2, 5, figsize=(18, 7))
        
        # Premiere ligne : image source + premieres frames
        axes[0][0].imshow(test_img)
        axes[0][0].set_title("Image source", fontsize=9, fontweight='bold')
        axes[0][0].axis('off')
        
        for i in range(1, 5):
            idx = i - 1
            if idx < len(i2v_frames):
                axes[0][i].imshow(i2v_frames[idx])
                axes[0][i].set_title(f"Frame {idx + 1}", fontsize=9)
            axes[0][i].axis('off')
        
        # Deuxieme ligne : frames suivantes
        later_indices = np.linspace(4, len(i2v_frames) - 1, 5, dtype=int)
        for i, idx in enumerate(later_indices):
            axes[1][i].imshow(i2v_frames[idx])
            axes[1][i].set_title(f"Frame {idx + 1}", fontsize=9)
            axes[1][i].axis('off')
        
        plt.suptitle("Image-to-Video : Animation d'un paysage", fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        if save_as_mp4:
            mp4_path = OUTPUT_DIR / "ltx_img2vid.mp4"
            export_to_video(i2v_frames, str(mp4_path), fps=fps_output)
            print(f"  MP4 sauvegarde : {mp4_path.name}")
        
        # Liberation du pipeline i2v
        del pipe_i2v
        if device == "cuda":
            torch.cuda.empty_cache()
        
    except ImportError as e:
        print(f"Pipeline image-to-video non disponible : {e}")
        print("Verifiez la version de diffusers (>=0.32 requise)")
    except Exception as e:
        print(f"Erreur image-to-video : {type(e).__name__}: {str(e)[:200]}")
else:
    print("Image-to-video desactive")
    print("\nPrincipe : LTXImageToVideoPipeline prend une image PIL + prompt")
    print("et genere une video qui anime l'image de maniere coherente.")

### Interpretation : Image-to-video

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Fidelite a l'image source | Elevee | La premiere frame est proche de l'image d'entree |
| Mouvement genere | Guide par le prompt | "clouds moving", "grass swaying" orientent le mouvement |
| VRAM | Similaire au text-to-video | L'image d'entree ne change pas significativement la consommation |

**Points cles** :
1. L'image-to-video est utile pour animer des photos, illustrations ou images generees par DALL-E / SDXL
2. Le prompt doit decrire le mouvement souhaite, pas juste la scene statique
3. Les mouvements subtils (nuages, eau, vent) donnent les meilleurs resultats

## Section 4 : Comparaison vitesse vs qualite

Nous allons comparer differentes configurations de LTX-Video pour trouver le meilleur
compromis entre vitesse de generation et qualite du resultat.

In [ ]:
# Comparaison vitesse vs qualite
if run_generation and pipe_t2v is not None:
    print("\n--- COMPARAISON VITESSE VS QUALITE ---")
    print("=" * 45)
    
    test_prompt = "a kitten playing with a ball of yarn, cute, soft lighting, cozy room"
    
    configs = [
        {"steps": 10, "cfg": 5.0, "label": "Rapide (10 steps)"},
        {"steps": 20, "cfg": 7.5, "label": "Equilibre (20 steps)"},
        {"steps": 40, "cfg": 7.5, "label": "Qualite (40 steps)"},
    ]
    
    speed_results = []
    
    for cfg in configs:
        print(f"\nTest : {cfg['label']}")
        
        # Modifier temporairement
        orig_steps = num_inference_steps
        orig_cfg = guidance_scale
        num_inference_steps = cfg['steps']
        guidance_scale = cfg['cfg']
        
        result = generate_ltx_video(test_prompt, seed=42)
        
        # Restaurer
        num_inference_steps = orig_steps
        guidance_scale = orig_cfg
        
        if result['success']:
            speed_results.append({
                "label": cfg['label'],
                "steps": cfg['steps'],
                "frames": result['frames'],
                "time": result['generation_time'],
                "vram_peak": result.get('vram_peak', 0)
            })
            print(f"  Temps : {result['generation_time']:.1f}s")
        else:
            print(f"  Erreur : {result['error']}")
    
    # Affichage comparatif
    if speed_results:
        n_configs = len(speed_results)
        n_preview = 4
        fig, axes = plt.subplots(n_configs, n_preview, figsize=(3.5 * n_preview, 3 * n_configs))
        if n_configs == 1:
            axes = [axes]
        
        for v_idx, sr in enumerate(speed_results):
            frame_indices = np.linspace(0, len(sr['frames']) - 1, n_preview, dtype=int)
            for f_idx, fi in enumerate(frame_indices):
                axes[v_idx][f_idx].imshow(sr['frames'][fi])
                axes[v_idx][f_idx].axis('off')
                if f_idx == 0:
                    axes[v_idx][f_idx].set_ylabel(sr['label'], fontsize=10, fontweight='bold')
        
        plt.suptitle("Vitesse vs Qualite - LTX-Video", fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Tableau recapitulatif
        print(f"\n{'Configuration':<25} {'Steps':<8} {'Temps (s)':<12} {'VRAM (GB)':<12}")
        print("-" * 57)
        for sr in speed_results:
            print(f"  {sr['label']:<25} {sr['steps']:<8} {sr['time']:<12.1f} {sr['vram_peak']:<12.1f}")
else:
    print("Comparaison vitesse/qualite : generation desactivee")
    print("\nGuide vitesse/qualite LTX-Video :")
    print("  10 steps : ~5s, preview rapide")
    print("  20 steps : ~12s, usage general")
    print("  40 steps : ~25s, meilleure qualite")

### Interpretation : Vitesse vs qualite

| Configuration | Temps | Qualite | Cas d'usage |
|--------------|-------|---------|-------------|
| 10 steps | ~5s | Acceptable | Preview rapide, prototypage |
| 20 steps | ~12s | Bonne | Usage general, iterations |
| 40 steps | ~25s | Tres bonne | Resultat final, production |

**Points cles** :
1. Le temps de generation est approximativement proportionnel au nombre de steps
2. Au-dela de 40 steps, le gain de qualite est marginal
3. Meme a 10 steps, LTX-Video produit des resultats utilisables

## Bonnes pratiques et comparaison

### Quand utiliser LTX-Video vs HunyuanVideo

| Scenario | Modele recommande | Raison |
|----------|------------------|--------|
| Prototypage rapide | LTX-Video | 2-5x plus rapide, VRAM reduite |
| Qualite maximale | HunyuanVideo | Meilleure coherence et details |
| GPU 8-12 GB | LTX-Video | Seul modele qui tient en memoire |
| Animation d'image | LTX-Video | Mode image-to-video natif |
| Video longue (5s+) | HunyuanVideo | Meilleure coherence temporelle |
| Iterations multiples | LTX-Video | Plus rapide pour tester des prompts |

In [ ]:
# Mode interactif
if notebook_mode == "interactive" and not skip_widgets:
    print("\n--- MODE INTERACTIF ---")
    print("=" * 40)
    print("Entrez votre propre prompt pour generer une video LTX-Video.")
    print("(Laissez vide pour passer a la suite)")
    
    try:
        user_prompt = input("\nVotre prompt : ").strip()
        
        if user_prompt and run_generation and pipe_t2v is not None:
            print(f"\nGeneration en cours...")
            result_user = generate_ltx_video(user_prompt, seed=123)
            
            if result_user['success']:
                print(f"Generation reussie en {result_user['generation_time']:.1f}s")
                
                n_display = min(8, len(result_user['frames']))
                fig, axes = plt.subplots(1, n_display, figsize=(2.5 * n_display, 3))
                if n_display == 1:
                    axes = [axes]
                indices = np.linspace(0, len(result_user['frames']) - 1, n_display, dtype=int)
                for ax, idx in zip(axes, indices):
                    ax.imshow(result_user['frames'][idx])
                    ax.set_title(f"Frame {idx+1}", fontsize=8)
                    ax.axis('off')
                plt.suptitle(f"Votre video : {user_prompt[:50]}...", fontweight='bold')
                plt.tight_layout()
                plt.show()
                
                if save_as_mp4:
                    user_mp4 = OUTPUT_DIR / "user_generation.mp4"
                    export_to_video(result_user['frames'], str(user_mp4), fps=fps_output)
                    print(f"MP4 sauvegarde : {user_mp4.name}")
            else:
                print(f"Erreur : {result_user['error']}")
        elif user_prompt:
            print("Generation non disponible (pipeline non charge)")
        else:
            print("Mode interactif ignore")
    
    except (KeyboardInterrupt, EOFError) as e:
        print(f"\nMode interactif interrompu ({type(e).__name__})")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("\nMode interactif non disponible (execution automatisee)")
        else:
            print(f"\nErreur inattendue : {error_type} - {str(e)[:100]}")
            print("Passage a la suite du notebook")
else:
    print("\nMode batch - Interface interactive desactivee")

In [ ]:
# Statistiques de session et prochaines etapes
print("\n--- STATISTIQUES DE SESSION ---")
print("=" * 40)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Modele : {model_id}")
print(f"Device : {device}")
print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
print(f"Resolution : {width}x{height}")

if device == "cuda" and torch.cuda.is_available():
    vram_peak = torch.cuda.max_memory_allocated(0) / 1024**3
    print(f"VRAM pic session : {vram_peak:.1f} GB")

if save_results and OUTPUT_DIR.exists():
    generated_files = list(OUTPUT_DIR.glob('*'))
    print(f"\nFichiers generes ({len(generated_files)}) :")
    for f in sorted(generated_files):
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")

# Liberation VRAM
if pipe_t2v is not None:
    del pipe_t2v
    if device == "cuda":
        torch.cuda.empty_cache()
        print(f"\nVRAM liberee")

print(f"\n--- PROCHAINES ETAPES ---")
print(f"1. Notebook 02-3 : Wan 2.1/2.2 (prompts multilingues FR/EN, motion control)")
print(f"2. Notebook 02-4 : SVD (animation d'images statiques haute qualite)")
print(f"3. Module 03-1 : Comparaison benchmark de tous les modeles video")
print(f"4. Combiner LTX image-to-video avec 01-4 (Real-ESRGAN) pour upscaling")

print(f"\nNotebook 02-2 LTX-Video Lightweight termine - {datetime.now().strftime('%H:%M:%S')}")